In [1]:
import pandas as pd

In [2]:
orderdf = pd.read_csv('./01Feb2021/NIFTY21FEBFUT_1.CSV')

In [3]:
#https://www.youtube.com/watch?v=oqUR41U-xT4&list=WL&index=11

In [4]:
orderdf.columns = ['Timestamp', 'TIME', 'Ord_ID1', 'Ord_ID2', 'Price', 'Qty',\
       'MessageType', 'LTP', 'LTQ', 'VTT', 'ORDER TYPE', 'BIDQ1', 'BID1',\
       'ASK1', 'ASKQ1', 'BIDQ2', 'BID2', 'ASK2', 'ASKQ2', 'BIDQ3', 'BID3',\
       'ASK3', 'ASKQ3', 'BIDQ4', 'BID4', 'ASK4', 'ASKQ4', 'BIDQ5', 'BID5',\
       'ASK5', 'ASKQ5', 'last']
orderdf.index.name = 'Sqn'

orderdf.head()

,Timestamp,TIME,Ord_ID1,Ord_ID2,Price,Qty,MessageType,LTP,LTQ,VTT,...,ASKQ3,BIDQ4,BID4,ASK4,ASKQ4,BIDQ5,BID5,ASK5,ASKQ5,last
Sqn,,,,,,,,,,,,,,,,,,,,,
10,1296638100010004196,09:15:00.010004,1100000000000919,0,13847.00,300,NEW ORDER,0.0,0,0,...,3000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,1296638100010241320,09:15:00.010241,1100000000000968,0,15080.05,150,NEW ORDER,0.0,0,0,...,3000,675.0,12338.20,13962.4,900.0,NaN,NaN,14525.25,600.0,NaN
82,1296638100010272855,09:15:00.010272,1100000000000974,0,13825.00,225,NEW ORDER,0.0,0,0,...,300,675.0,12338.20,13869.9,3000.0,NaN,NaN,13962.40,900.0,NaN
83,1296638100010277142,09:15:00.010277,1100000000000975,0,13785.00,75,NEW ORDER,0.0,0,0,...,300,600.0,12825.55,13869.9,3000.0,675.0,12338.20,13962.40,900.0,NaN
86,1296638100010292448,09:15:00.010292,1100000000000978,0,13451.25,900,NEW ORDER,0.0,0,0,...,300,1800.0,13451.25,13869.9,3000.0,600.0,12825.55,13962.40,900.0,NaN


In [4]:
#orderdf.info()
#orderdf['TIME2'] = pd.to_datetime(orderdf['TIME'], infer_datetime_format= True)
#orderdf = orderdf.drop(['TIME2'], axis=1)
#from datetime import datetime
#datetime.now()

In [5]:
#pd.to_datetime(orderdf['TIME'], format='%H:%M:%S.%f')

In [5]:
orderdf['TIME'] = pd.to_datetime(orderdf['TIME'], format='%H:%M:%S.%f')
orderdf = orderdf.set_index(['TIME'])

**PTR**: Consider rectifying the date as 01-01-1900. This needs to be corrected. I am trying to find a way to specify the exact date. 

In [7]:
price_ohlc = orderdf['Price'].resample('60s').ohlc()
price_ohlc

,open,high,low,close
TIME,,,,
1900-01-01 09:15:00,13847.00,15080.05,13451.25,13746.95
1900-01-01 09:16:00,13752.65,13770.50,13747.20,13762.45
1900-01-01 09:17:00,13762.50,13780.95,13760.15,13774.90
1900-01-01 09:18:00,13774.95,13784.90,13767.15,13777.20
1900-01-01 09:19:00,13777.90,13781.80,13752.65,13758.40
1900-01-01 09:20:00,13759.90,13759.90,13715.00,13733.70
1900-01-01 09:21:00,13733.70,13744.45,13718.05,13734.80
1900-01-01 09:22:00,13734.85,13764.40,13729.10,13746.25
1900-01-01 09:23:00,13746.45,13772.75,13745.95,13763.30


In [8]:
"""
bids = orderdf['BID1'].resample('60s').ohlc()
bids

asks = orderdf['ASK2'].resample('60s').ohlc()
asks
"""

"\nbids = orderdf['BID1'].resample('60s').ohlc()\nbids\n\nasks = orderdf['ASK2'].resample('60s').ohlc()\nasks\n"

### Total successful trades

In [13]:
tradedf = orderdf[orderdf.MessageType == 'TRADE']
successful_trades = tradedf['MessageType'].resample('1min').count()
cancelled_trades = orderdf[orderdf.MessageType == 'CANCELLATION']['MessageType'].resample('1min').count()
new_orders = orderdf[orderdf.MessageType == 'NEW ORDER']['MessageType'].resample('1min').count()
modified_df = orderdf[orderdf.MessageType == 'MODIFICATION']['MessageType'].resample('1min').count()

In [18]:
df = pd.DataFrame()
df = pd.concat([df, new_orders, cancelled_trades, successful_trades, modified_df], axis=1)
df.columns = ['neworders', 'cancelled_orders', 'trades', 'modified']
df

,neworders,cancelled_orders,trades,modified
1900-01-01 09:15:00,8422.0,5367.0,3389.0,25225
1900-01-01 09:16:00,4574.0,3220.0,1488.0,19869
1900-01-01 09:17:00,3561.0,2434.0,1310.0,12168
1900-01-01 09:18:00,2578.0,2007.0,802.0,10462
1900-01-01 09:19:00,2588.0,1916.0,798.0,8395
1900-01-01 09:20:00,10289.0,7037.0,1934.0,41652
1900-01-01 09:21:00,4472.0,3441.0,665.0,16954
1900-01-01 09:22:00,4667.0,3742.0,670.0,16579
1900-01-01 09:23:00,4971.0,4004.0,863.0,18324
1900-01-01 09:24:00,3650.0,2888.0,462.0,11732


### Market Orders
This can be detected by comparing price with the Ask. If price == Ask then it's a market order. 

In [41]:
market_orders = orderdf[((orderdf['ORDER TYPE'] == 'BUY') & (orderdf['Price'] >= orderdf['ASK1'])) \
                        | ((orderdf['ORDER TYPE'] == 'SELL') & (orderdf['Price'] <= orderdf['BID1']))]

In [43]:
market_orders.MessageType.unique()

array(['NEW ORDER', 'CANCELLATION', 'TRADE', 'MODIFICATION'], dtype=object)

In [44]:
market_orders= market_orders[market_orders['MessageType'] !='CANCELLATION']
market_orders_df = market_orders['ORDER TYPE'].resample('1min').count()

In [49]:
df = df.join(market_orders_df)
df.rename(columns = {'ORDER TYPE': 'market_orders'}, inplace = True)

In [50]:
df

,neworders,cancelled_orders,trades,modified,no_market,market_orders
1900-01-01 09:15:00,8422.0,5367.0,3389.0,25225,1043.0,1043.0
1900-01-01 09:16:00,4574.0,3220.0,1488.0,19869,407.0,407.0
1900-01-01 09:17:00,3561.0,2434.0,1310.0,12168,380.0,380.0
1900-01-01 09:18:00,2578.0,2007.0,802.0,10462,273.0,273.0
1900-01-01 09:19:00,2588.0,1916.0,798.0,8395,299.0,299.0
1900-01-01 09:20:00,10289.0,7037.0,1934.0,41652,435.0,435.0
1900-01-01 09:21:00,4472.0,3441.0,665.0,16954,194.0,194.0
1900-01-01 09:22:00,4667.0,3742.0,670.0,16579,235.0,235.0
1900-01-01 09:23:00,4971.0,4004.0,863.0,18324,281.0,281.0
1900-01-01 09:24:00,3650.0,2888.0,462.0,11732,135.0,135.0
